# Мэтчинг товаров

Цель проекта: Подобрать и обучить модель на исходных данных, способную найти 5 похожих товаров для валиадационной выборки из датасета base, основываясь на метрики accuracy@5.

Задачи:
- Загрузка датасетов и предварительный обзор;
- EDA;
- Построение Baseline-моделей и выбор наилучшего варианта;
- Предобработка данных перед обучением;
- Обучение модели и анализ результатов

Интерументы: В проекте использовались алгоритмы реализованные в библиотеки FAISS, обучение происходило на GPU.

In [ ]:
# %conda install -c pytorch faiss-cpu (если нет CUDA)
%conda install -c conda-forge faiss-gpu

In [ ]:
%conda install -c "conda-forge/label/broken" faiss-gpu

## Импорты и константы

### Импорты

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.express as px
import faiss
import sweetviz as sv
import numpy as np
from sklearn.preprocessing import RobustScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

### Константы

In [ ]:
k_similar = 5 # Количество (соседей) похожих товаров

## Загрузка и обзор данных

In [ ]:
train = pd.read_csv('./Data/train.csv', index_col=0) 
valid = pd.read_csv('./Data/validation.csv', index_col=0)
valid_awr = pd.read_csv('./Data/validation_answer.csv', index_col=0)
base = pd.read_csv('./Data/base.csv', index_col=0)

In [ ]:
train.describe().T\
    .style.bar(subset=['mean'], color=px.colors.qualitative.G10[2])\
    .background_gradient(subset=['std'], cmap='Blues')\
    .background_gradient(subset=['50%'], cmap='BuGn')

In [ ]:
base.describe().T\
    .style.bar(subset=['mean'], color=px.colors.qualitative.G10[2])\
    .background_gradient(subset=['std'], cmap='Blues')\
    .background_gradient(subset=['50%'], cmap='BuGn')

Сразу можно обратить внимание, что есть некоторые признаки, которые имеют сильный разброс по стандартному отклонению, этот фактор может являтся ключевым при обучении модели.

Датасет: Набор даннх имеет 72 признака, в Base имеется порядка 3 млн. записей, пропуски и дубликаты в данных отсутсвуют. Данные представленны ввиде вещественных чисел Float64.

In [ ]:
d = base.shape[1] # Получим количество признаков
d

In [ ]:
features_t = train.drop('Target', axis=1) # Из датасета Train возьмем признаки

Выполним явное преобразование типов данных в датасетах, для уменьшения занимаемого пространства

In [ ]:
features_t = features_t.astype('float32')
valid = valid.astype('float32')
base = base.astype('float32')


In [ ]:
ngpu = 2 # Количество видеокарт

## EDA

Воспользуемся библиотекой `sweetviz` для быстрого анализа данных

In [ ]:
train_report = sv.analyze(base)

In [ ]:
train_report.show_html('Base_report.html')

## Baseline (FAISS)

Для решения задачи метчинага была выбрана одна из наиболее эффективных и популярных библитек от Facebook - FAISS. 

В работе исследованы модели `FlatL2` `IVF` `HNSW`.

### Flat L2

In [ ]:
index = faiss.IndexFlatL2(d)
resources = [faiss.StandardGpuResources() for i in range(ngpu)]
index_gpu = faiss.index_cpu_to_gpu_multiple_py(resources, index)
index_gpu.add(base)

In [ ]:
base_index = {k: v for k, v in enumerate(base.index.to_list())}

In [ ]:
targets = train["Target"]

In [ ]:
targets_v = valid_awr['Expected']

In [ ]:
%%time
D, I = index_gpu.search(features_t, k_similar)
acc = 0
for target, el in zip(targets.values.tolist(), I.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(I))

In [ ]:
%%time
D, I = index_gpu.search(valid, k_similar)
acc = 0
for target, el in zip(targets_v.values.tolist(), I.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(I))

## IVF

In [ ]:
nlist = 18

quant = faiss.IndexFlatIP(d)
index = faiss.IndexIVFFlat(quant, d, nlist)

resources = [faiss.StandardGpuResources() for i in range(ngpu)]
index_gpu = faiss.index_cpu_to_gpu_multiple_py(resources, index)

index_gpu.train(base)
index_gpu.add(base)
index_gpu.nprobe = 8

In [ ]:
%%time
D, I = index_gpu.search(features_t, k_similar)
acc = 0
for target, el in zip(targets.values.tolist(), I.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(I))

In [ ]:
%%time
D, I = index_gpu.search(valid, k_similar)
acc = 0
for target, el in zip(targets_v.values.tolist(), I.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(I))

## HNSW

In [ ]:
index = faiss.IndexHNSWFlat(d, 1024)
resources = [faiss.StandardGpuResources() for i in range(ngpu)]
index_gpu = faiss.index_cpu_to_gpu_multiple_py(resources, index)
index_gpu.add(base)

In [ ]:
%%time
D, I = index_gpu.search(features_t, k_similar)
acc = 0
for target, el in zip(targets.values.tolist(), I.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(I))

In [ ]:
%%time
D, I = index_gpu.search(valid, k_similar)
acc = 0
for target, el in zip(targets_v.values.tolist(), I.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(I))

### Результаты baseline

**Выбор модели**

Были рассмотрены три модели `FlatL2` `IVF` и `HNSW`. Для опеределения наилучшей модели была использована метрика accuracy@5 (среднее значение accuracy для 5 метчей).


**Обучение моделей**

Показатели метрик для валидационной выборки:
- `FlatL2`
    - Accuracy@5 `13.286`
- `IVF`
    - Accuracy@5 `13.155`
- `HNSW`
    - Accuracy@5 `11.431`

Далее будет использоваться модель **FlatL2**

## Предобработка данных

Проверим данные на нормальность (подчинению закону нормального расспределения); Проведем тест Шапиро-Уилка; Оценим параметр ассиметрии данных (skew); Выполним масштабирование данных.

In [ ]:
base.hist(figsize=[20, 20], bins=50);

### Тест Шапиро-Улка на нормальность данных

Отметим только те признаки которые попадают в интервал 3-х sigm, на небольшом отрезке данных (При величине вектора больше 5000, p-value будет работать нестабильно) 

In [ ]:
from scipy.stats import shapiro 

def statistic(x):
    return shapiro(x).statistic

for i in base:
    if statistic(base[i][:5000]) <= 0.99:
        print(i, statistic(base[i][:5000]))

### SKEW

In [ ]:
def summary(df):
    sum = pd.DataFrame(df.dtypes, columns=['dtypes'])
    sum['count'] = df.count().values
    sum['skew'] = df.skew().values
    return sum

s = summary(base)
s.style.background_gradient(cmap='Blues')

In [ ]:
s.query('skew > 0.5 or skew < -0.5')

Уберем из наших исходных данных признаки которые не подчиняются закону нормального расспределения

In [ ]:
base_norm = base.drop(['6','21','25','33','44','59','63','65','70'], axis=1)
train_norm = train.drop(['6','21','25','33','44','59','63','65','70'], axis=1)
valid_norm = valid.drop(['6','21','25','33','44','59','63','65','70'], axis=1)

In [ ]:
features_t_norm = train_norm.drop('Target', axis=1)

### Мультиколлениарность

Выполним проверку на мультиколлениарность данных

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = base_norm.columns

# вычисление VIF для каждого признака
vif_data["VIF"] = [variance_inflation_factor(base_norm.values, i)
                          for i in range(len(base_norm.columns))]
  
print(vif_data)

In [ ]:
VIF_features_drop = vif_data.query('VIF > 9.0')['feature']
VIF_features_drop

In [ ]:
base_vif = base_norm.drop(VIF_features_drop.values, axis=1)
train_vif = train_norm.drop(VIF_features_drop.values, axis=1)
valid_vif = valid_norm.drop(VIF_features_drop.values, axis=1)

In [ ]:
features_t_vif = train_vif.drop('Target', axis=1)

In [ ]:
base_vif.hist(figsize=[20, 20], bins=50);

### Масштабирование 

In [ ]:
scaler = RobustScaler()
base_slr_vif = scaler.fit_transform(base_vif)
features_slr_vif = scaler.transform(features_t_vif)
valid_slr_vif = scaler.transform(valid_vif)

## Обучение модели

In [ ]:
d_vif = base_slr_vif.shape[1]

In [ ]:
index = faiss.IndexFlatL2(d_vif)
index.add(base_slr_vif)

In [ ]:
resources = [faiss.StandardGpuResources() for i in range(ngpu)]
index_gpu = faiss.index_cpu_to_gpu_multiple_py(resources, index)

In [ ]:
base_vif_index = {k: v for k, v in enumerate(base_vif.index.to_list())}

In [ ]:
%%time
D, I = index_gpu.search(features_slr_vif, k_similar)
acc = 0
for target, el in zip(target_vif.values.tolist(), I.tolist()):
    acc += int(target in [base_vif_index[r] for r in el])

print(100 * acc / len(I))

In [ ]:
%%time
D, I = index_gpu.search(valid_slr_vif, k_similar)
acc = 0
for target, el in zip(targets_v.values.tolist(), I.tolist()):
    acc += int(target in [base_vif_index[r] for r in el])

print(100 * acc / len(I))

## Выводы

Цель проекта было подобрать и обучить модель на исходных данных, способную найти 5 похожих товаров для валиадационной выборки из датасета base, основываясь на метрики accuracy@5.

Выполненые задачи:
- Загрузка датасетов и предварительный обзор;
- EDA;
- Построение Baseline-моделей и выбор наилучшего варианта;
- Предобработка данных перед обучением;
- Обучение модели и анализ результатов

Для решения задачи метчинага была выбрана одна из наиболее эффективных и популярных библитек от Facebook - FAISS. 

В работе исследованы модели `FlatL2` `IVF` и `HNSW`.

Показатели метрик для валидационной выборки:
- `FlatL2`
    - Accuracy@5 `13.286`
- `IVF`
    - Accuracy@5 `13.155`
- `HNSW`
    - Accuracy@5 `11.431`

Далее использовалась модель **FlatL2**

Итоговые результаты для валидационной выборки:
- `FlatL2`
    - Accuracy@5 `69.569`